In [1]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 32.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Prepari

In [2]:
import ktrain
from ktrain import text

In [3]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [4]:
DATA_PATH = '/content/train.csv'
NUM_WORDS = 500000
MAXLEN = 500
trn ,val , preproc   = text.texts_from_csv( DATA_PATH,
                          'text',
                          label_columns = 'sentiment',
                          val_filepath=None, # if None, 10% of data will be used for validation
                          max_features=NUM_WORDS, maxlen=MAXLEN,
                          ngram_range=1, 
                          preprocess_mode='distilbert')

detected encoding: utf-8 (if wrong, set manually)
['not_sentiment', 'sentiment']
       not_sentiment  sentiment
9480             1.0        0.0
3934             1.0        0.0
10579            1.0        0.0
9906             1.0        0.0
11784            1.0        0.0
['not_sentiment', 'sentiment']
      not_sentiment  sentiment
4930            1.0        0.0
6890            1.0        0.0
49              1.0        0.0
5364            1.0        0.0
4233            1.0        0.0


preprocessing train...
language: en
train sequence lengths:
	mean : 231
	95percentile : 592
	99percentile : 907


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 594
	99percentile : 889


In [5]:
x_train = trn.x
y_train = trn.y
x_test = val.x
y_test = val.y

In [6]:
x_train.data

In [7]:
x_train.shape , x_test.shape

((22500, 3, 500), (2500, 3, 500))

In [8]:
model = text.text_classifier('distilbert', trn, preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [9]:
learner = ktrain.get_learner(model, 
                             trn,
                             val,
                             batch_size=12)

In [10]:
learner.fit_onecycle(1e-4, 1)



begin training using onecycle policy with max lr of 0.0001...
1875/1875 [==============================] - 1360s 717ms/step - loss: 0.2886 - accuracy: 0.8802 - val_loss: 0.1885 - val_accuracy: 0.9292


In [11]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [12]:
data = [ 'This movie was horrible! The plot was boring. Acting was okay, though.',
         'The film really sucked. I want my money back.',
        'What a beautiful romantic comedy. 10/10 would see again!']

In [13]:
predictor.predict(data)

['sentiment', 'sentiment', 'not_sentiment']

In [14]:
import pandas as pd
df = pd.read_csv('/content/train.csv')
df.head()

,text,sentiment
0,For a movie that gets no respect there sure ar...,0
1,Bizarre horror movie filled with famous faces ...,0
2,"A solid, if unremarkable film. Matthau, as Ein...",0
3,It's a strange feeling to sit alone in a theat...,0
4,"You probably all already know this by now, but...",0


In [15]:
selected_rows = df[12000:13000]
selected_text = selected_rows['text']
selected_sentiment = selected_rows['sentiment']
selected_text ,selected_sentiment

(12000    So, neighbor was killing neighbor. Reminds me ...
 12001    The first time i saw it i got half of it but i...
 12002    This has to be one of the best comedies on the...
 12003    This would have to be one of the funniest TV s...
 12004    I gave this a 10 out of 10 points. I love it s...
                                ...                        
 12995    I just finished watching this film and WOW was...
 12996    The script for this TV soap opera is so bad th...
 12997    In 1987, John Hughes wrote and directed 'Plane...
 12998    The NSA, CIA, FBI, FSB and all other snoop age...
 12999    What on earth has become of our dear Ramu? Is ...
 Name: text, Length: 1000, dtype: object,
 12000    0
 12001    0
 12002    0
 12003    0
 12004    0
         ..
 12995    1
 12996    1
 12997    1
 12998    1
 12999    1
 Name: sentiment, Length: 1000, dtype: int64)

In [16]:
max(selected_sentiment) , min(selected_sentiment)

(1, 0)

In [17]:
selected_sentiment.value_counts()

0    500
1    500
Name: sentiment, dtype: int64

In [18]:
y_pred = predictor.predict(selected_text.tolist(), verbose=0)

In [19]:
#import numpy as np
#y_pred_d = np.round(y_pred) 
mapping = {'not_sentiment':0 , 'sentiment':1}
y_pred_d = list(mapping[val] for val in y_pred )


In [20]:
predictor.get_classes()

['not_sentiment', 'sentiment']

In [21]:
#Classification report for bag of words 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
report =classification_report(selected_sentiment,y_pred_d)#,target_names=['Positive','Negative'])
print(report)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       500
           1       0.98      0.96      0.97       500

    accuracy                           0.97      1000
   macro avg       0.97      0.97      0.97      1000
weighted avg       0.97      0.97      0.97      1000



In [22]:
predictor.save('/content/bert_model')

In [26]:
print('model saved')

model saved
